# Tester notebook

This notebook aims to create enviroment to simply test strategies developed in Stocks & Loss system for python

In [ ]:
# imports 
from scraper import stock_daily
from analyzer import Analyzer
import numpy as np
import pandas as pd
from IPython.display import display

## Load stocks across industries

In [ ]:
xls = pd.ExcelFile('TickerPerSector.xls')
fundTickers = pd.DataFrame()
Tickers = pd.DataFrame()
fundTickers["Fund/sector name"] = xls.sheet_names
# Tickers = pd.DataFrame(,columns=funTickers["Index fund name"])
for i in range(len(xls.sheet_names)):
    sheet1 = pd.read_excel(xls, i)
    fundTickers.loc[i,["Index fund ticker"]] = sheet1.columns[1]
    Tickers[fundTickers.loc[i,["Fund/sector name"]]] = sheet1[sheet1.columns[1]][1:]
pd.set_option('display.expand_frame_repr', False)    
display(Tickers)

## Iterate over each industry

In [ ]:
# iterate over columns
i = 0 
results = []
for label, tic in Tickers.iteritems():
    results.append(pd.DataFrame(columns=["Ticker", "Trades", "Good trades", "Profit[%]", "Profit[$]", "Hold[%]"]))
    print(label)
    # iterate over items in sectors / indexes
    for j, val in tic.iteritems():
        # analyze stock
        try:
            anal = Analyzer(ticker=val,data=stock_daily(val, save=False).data)
            anal.strategy(buyStrategy=['Mcstoch_ut1','Mcstoch_ut2', 'Mcstoch_ut3', 'Mcstoch_ut4'],sellStrategy=['Mcstoch'],stopLoss=True,stopLossValue=0.05,profitTaker=True,profitTakerValue=0.1)
            ts = anal.profit(capitalForEachTrade=400,comission=2)
            # save data into dataframe
            results[i].loc[j,["Ticker"]] = val
            results[i].loc[j,["Trades"]] = ts.shape[0]
            results[i].loc[j,["Good trades"]] = ts["Good trade?"].sum()
            results[i].loc[j,["Profit[%]"]] = ts["Profit[%]"].sum()
            results[i].loc[j,["Profit[$]"]] = ts["Profit[%]"].sum()
            results[i].loc[j,["Hold[%]"]] = 100*((anal.data["Close"].iloc[-1]-anal.data["Close"].iloc[0])/anal.data["Close"].iloc[0])
        except:
            results[i].loc[j,["Ticker"]] = val
    i += 1

In [ ]:
display(results[2])